In [ ]:
import pandas as pd

data_path= 'C:\\Users\\c.hernandezperez\\Desktop\\drop\\reports\\PRSA_Ireland-Pensions-sit_2019-01-29 16-24.html'
#prueba
df = pd.read_html(data_path)
quit()
#begin_test_condition = df[0]['Details'].iloc[0]
#print(begin_test_condition)
display(df)
# display(df)
# print(type(df))

In [ ]:
#display(df[0]['Status'].iloc[-1])# print(len(df))
print(type(df[0]))
display(df[0])



# print(df[0].Status.value_counts()30
# 'cancel' in df[30]['Status'].tolist()

In [6]:
from bs4 import BeautifulSoup

'''
Workflow
1- Sumar pasos
2 - Checkear si el ultimo es un cancel
   2.1 - Si es un cancel, guardar la informacion en la primera fila del DF_N
   2.2 - Si no es un cancel, checkear si el test acabo
       2.2.1 - Si aun no acabo sumar los pasos a los que habia antes
       2.2.2 - Guardar los datos en el DF y comenzar una fila nueva
       
       
TODO

Hacer cambios para poder trabajar con varios workflows

'''
def extract_info(df_path):
    print(df_path)
    try:
        df = pd.read_html(df_path)
    except:
        data_path= 'C:\\Users\\c.hernandezperez\\Desktop\\drop\\reports\\PRSA_Ireland-Pensions-sit_2019-01-29 12-54.html'
        soup = BeautifulSoup(open(data_path), "html.parser")
        #prueba
        df = pd.read_html(str(soup.find_all('table')))
    # On this list we will be adding our dictionaries to improve speed
    rows_list = []
    steps = 0
    start_time=0
    end_time=0
    try:
        print(df[0]['Details'].iloc[0])
    except:
        import pdb;pdb.set_trace()
        pass
    
    # This dictionary will be the one filling the rows_list
    dict_test = {}
    df_extracted = pd.DataFrame(columns=['Steps','Status','Start Time', 'End Time'])
    # Boolean to check if we are still on a test case
    in_test = False


    for i in range(len(df)-1):
        #Starting a test case
     #   import pdb;pdb.set_trace()
        # This if statement is the weakest part, should modify for multi work-flows
      #  import pdb;pdb.set_trace()
        
        # It's <=2 because some reports had only 1
        if len(df[i])==2 and df[i]['Timestamp'].iloc[0]==df[i]['Timestamp'].iloc[-1] and in_test==False:
            print('entered in the interlude')
            #import pdb; pdb.set_trace()
            pass
        else:
            # This if statement is the weakest part, should modify for multi work-flows
            if "Loaded the URL" in df[i]['Details'].iloc[0] and df[i]['Status'].sum()!=1:
                print('first time it enters in a test')
               # import pdb;pdb.set_trace()
                 # If we are on one work-flow this will make sure to save the previous test case
                # before getting on the next one
                if in_test:
                    print('Saved previous test')
                    #import pdb;pdb.set_trace()
                    status = 'pass'
                    #import pdb; pdb.set_trace()
                    # Add a new row
                    dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                    rows_list.append(dict_test)
                    dict_test = {}
                    in_test = False


              #  import pdb;pdb.set_trace()
                # If we have a fail on the first mayor step, it will stop here and save the test in the list
                if ('cancel' or 'warning') in df[i]['Status'].tolist():
                    print('first time it enters but itis a fail')
                    # Values of the test
                    steps = df[i].Status.value_counts()['info_outline']
                    start_time = df[i]['Timestamp'].iloc[0]
                    end_time   = df[i]['Timestamp'].iloc[-1]
                    status = 'fail'
                
                    # Add a new row
                    dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                    rows_list.append(dict_test)
                    dict_test = {}
                    in_test = False




                # Recopilar pasos
                # The first time it enters a test
                else:
                    start_time = df[i]['Timestamp'].iloc[0]
                    end_time = df[i]['Timestamp'].iloc[-1]
                    steps = df[i].Status.value_counts()['info_outline']
                    in_test = True

                    # If it is the same 
                    if i==(len(df)-2):
                        status = 'pass'
                        # Add a new row
                        dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                        rows_list.append(dict_test)
                        dict_test = {}


            #We will enter here if there is continuation on a text or if the test was fatal
            else:
                if df[i].Status.value_counts().sum()==1 and ('redo' in df[i]['Status'].tolist() or 'cancel' in df[i]['Status'].tolist()):
                   # import pdb;pdb.set_trace()
                    
                    if in_test:
                        status = 'pass'
                        # Add a new row
                        dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                        rows_list.append(dict_test)
                        dict_test = {}
                        in_test = False
                    
                    
                    # In this case we do not need the "tolist()" method
                    if 'cancel' in df[i].Status.value_counts():
                        status = 'fatal'
                    else:
                        status = 'skip'
                    steps = 1
                    start_time = df[i]['Timestamp'].iloc[0]
                    end_time   = df[i]['Timestamp'].iloc[0]

                    # Add a new row
                    dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                    rows_list.append(dict_test)
                    dict_test = {}
                else:
                    # Continuing the test case but it is a failed test
                    if 'cancel' in df[i]['Status'].tolist():
                        #print('seems to be working')
                      #  import pdb;pdb.set_trace()
                        try:
                            steps += df[i].Status.value_counts()['info_outline']
                        except:
                            pass
                        end_time = df[i]['Timestamp'].iloc[-1]
                        status = 'fail'

                        # Add a new row
                        dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                        rows_list.append(dict_test)
                        dict_test = {}
                        in_test = False

                    # Continuing the test case
                    else:
                       # print('continue on the test testing')
                        #import pdb;pdb.set_trace()
                        try:
                            steps += df[i].Status.value_counts()['info_outline']
                            end_time = df[i]['Timestamp'].iloc[-1]
                        except:
                            pass
                          #  import pdb;pdb.set_trace()
                        in_test = True
                        if i==(len(df)-2):
                           # import pdb;pdb.set_trace()
                            status = 'pass'
                            # Add a new row
                            dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                            rows_list.append(dict_test)
                            dict_test = {}

                        # Añadir al dataframe
    df_extracted = pd.DataFrame(rows_list)
    
    soup = BeautifulSoup(open(df_path), "html.parser")
    test_names = []

    for test in soup.find_all('span', {'class' : 'test-name'}):
        print(test.text)
        test_names.append(test.text)
    try:
        df_extracted['test_names'] = test_names
    except:
        
        # Add a new row
        status = 'pass'
        dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
        rows_list.append(dict_test)
        df_extracted = pd.DataFrame(rows_list)
        import pdb;pdb.set_trace()
        df_extracted['test_names'] = test_names
        
    return df_extracted
    # df_extraced = pd.concat([df_extracted, df_middle])

# display(df_middle)

In [7]:
import os
import pandas as pd

# Read all the file names of a given directory
data_path = 'C:\\Users\\c.hernandezperez\\Desktop\\drop\\reports\\' # Cambiar esto
dirs = os.listdir(data_path)
# Open all the excel files and concatenate the data into a single variable
df = pd.DataFrame()
for name in dirs:
    extracted = extract_info(data_path+name)
    display(extracted)

    df = pd.concat([df, extracted], axis=0)
df.to_excel(data_path+'prueba_2.xlsx',index=False)

C:\Users\c.hernandezperez\Desktop\drop\reports\338.html
LA_FranceDomestic
entered in the interlude
first time it enters in a test
entered in the interlude
first time it enters in a test
first time it enters in a test
entered in the interlude
first time it enters in a test
entered in the interlude
first time it enters in a test
first time it enters in a test
Saved previous test
LA_FranceDomestic
LA_GermanyDomesticLayered
LA_SingaporeIpzRenewal
LA_SwedenIpzMta
LA_SwitzerlandIpzLayered
LA_UkDomesticCancellation


,Steps,Status,Start Time,End Time,test_names
0,107,fail,10:23:54 AM,10:26:52 AM,LA_FranceDomestic
1,149,fail,10:26:54 AM,10:37:35 AM,LA_GermanyDomesticLayered
2,149,fail,10:31:13 AM,10:37:35 AM,LA_SingaporeIpzRenewal
3,164,fail,10:37:37 AM,10:44:38 AM,LA_SwedenIpzMta
4,164,pass,10:44:40 AM,10:49:30 AM,LA_SwitzerlandIpzLayered
5,119,pass,10:49:32 AM,10:52:34 AM,LA_UkDomesticCancellation


C:\Users\c.hernandezperez\Desktop\drop\reports\339.html
'SPC_BasicQuoteVerifyQuote' PASSED
entered in the interlude
first time it enters in a test
first time it enters in a test
Saved previous test
first time it enters in a test
Saved previous test
first time it enters in a test
Saved previous test
first time it enters in a test
Saved previous test
first time it enters in a test
Saved previous test
first time it enters in a test
Saved previous test
first time it enters in a test
Saved previous test
first time it enters in a test
Saved previous test
first time it enters in a test
Saved previous test
first time it enters in a test
entered in the interlude
first time it enters in a test
SPC_BasicQuoteVerifyQuote
SPC_EditQuoteCoverDefault
SPC_FxRatesOverride
SPC_ValidationsCyber
SPC_ValidationsCoversEndorsements
SPC_ValidationsNoCyber
SPC_CalculationReportedInXml
SPC_MultiVersionsVersionsFile
SPC_PolicyStateBound
SPC_PolicyStateMte
SPC_PolicyStateRenewal
SPC_PolicyStateCancellation


,Steps,Status,Start Time,End Time,test_names
0,118,pass,11:03:30 AM,11:06:35 AM,SPC_BasicQuoteVerifyQuote
1,97,pass,11:06:37 AM,11:09:03 AM,SPC_EditQuoteCoverDefault
2,101,pass,11:09:04 AM,11:12:33 AM,SPC_FxRatesOverride
3,166,pass,11:12:34 AM,11:21:44 AM,SPC_ValidationsCyber
4,255,pass,11:21:46 AM,11:27:30 AM,SPC_ValidationsCoversEndorsements
5,131,pass,11:27:32 AM,11:30:26 AM,SPC_ValidationsNoCyber
6,175,pass,11:30:28 AM,11:35:16 AM,SPC_CalculationReportedInXml
7,150,pass,11:35:18 AM,11:45:23 AM,SPC_MultiVersionsVersionsFile
8,46,pass,11:45:25 AM,11:46:54 AM,SPC_PolicyStateBound
9,44,fail,11:46:56 AM,11:50:55 AM,SPC_PolicyStateMte


C:\Users\c.hernandezperez\Desktop\drop\reports\341.html
'ML_UkMultiLine' PASSED
entered in the interlude
first time it enters in a test
first time it enters in a test
ML_UkMultiLine
ML_UkMultiLineRenewal


...

,Steps,Status,Start Time,End Time,test_names
0,17,fail,3:53:40 PM,3:54:59 PM,Group3_Prerequisite
1,12,fail,3:55:14 PM,3:56:19 PM,TC044_Verify_The_Currency_Value_of_The_Default...
2,12,fail,3:56:38 PM,3:57:41 PM,TC064_Verify_the_process_of_Creating_New_Time_...
3,12,fail,3:57:54 PM,3:59:00 PM,TC038_Verify_That_Change_In_The_FieldValue_Of_...
4,12,fail,3:59:14 PM,4:00:21 PM,TC004_Creation_Of_SprinklerSystems_TestDataSpr...
5,12,fail,4:00:38 PM,4:01:40 PM,TC039_Verify_That_Change_In_The_FieldValue_Of_...
6,12,fail,4:01:54 PM,4:03:01 PM,TC040_Verify_That_Change_In_The_FieldValue_Of_...
7,12,fail,4:03:14 PM,4:04:20 PM,TC009_Creation_of_Water_Supply_Systems_Test_Da...
8,12,fail,4:04:36 PM,4:05:41 PM,TC041_Verify_That_Change_In_The_FieldValue_Of_...
9,12,fail,4:05:54 PM,4:07:03 PM,TC005_Creation_of_Assignment_in_Assigned_status


C:\Users\c.hernandezperez\Desktop\drop\reports\index_fatal.html
java.lang.Exception: chrome Browser did not get launched ' 	at core.Reporter.logFATAL(Reporter.java:67) 	at core.WebFunctions.launchBrowser(WebFunctions.java:114) 	at reusuables.Web.doppoCarInsurance(Web.java:112) 	at scenarios.QuoteCreation.test(QuoteCreation.java:41) 	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method) 	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62) 	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43) 	at java.lang.reflect.Method.invoke(Method.java:498) 	at org.testng.internal.MethodInvocationHelper.invokeMethod(MethodInvocationHelper.java:124) 	at org.testng.internal.Invoker.invokeMethod(Invoker.java:583) 	at org.testng.internal.Invoker.invokeTestMethod(Invoker.java:719) 	at org.testng.internal.TestMethodWithDataProviderMethodWorker.call(TestMethodWithDataProviderMethodWorker.java:71) 	at org.testng.internal.TestMethodWith

,Steps,Status,Start Time,End Time,test_names
0,1,fatal,2:26:49 PM,2:26:49 PM,Test ID :- SALE_P_11
1,1,fatal,2:26:50 PM,2:26:50 PM,Test ID :- SALE_P_12
2,1,fatal,2:26:51 PM,2:26:51 PM,Test ID :- SALE_P_13
3,1,fatal,2:26:52 PM,2:26:52 PM,Test ID :- SALE_P_14
4,1,fatal,2:26:54 PM,2:26:54 PM,Test ID :- SALE_P_15
5,1,fatal,2:26:55 PM,2:26:55 PM,Test ID :- SALE_P_16
6,1,fatal,2:26:57 PM,2:26:57 PM,Test ID :- SALE_P_17
7,1,fatal,2:26:58 PM,2:26:58 PM,Test ID :- SALE_P_18
8,1,fatal,2:26:59 PM,2:26:59 PM,Test ID :- SALE_P_19
9,1,fatal,2:27:00 PM,2:27:00 PM,Test ID :- SALE_P_20


C:\Users\c.hernandezperez\Desktop\drop\reports\Regression_Suite-UAT_2020-02-25 07-14.html
Loaded the URL : 'https://owl-gatewaywebapp-uat.azurewebsites.net/contratar-seguro-coche'
first time it enters in a test
first time it enters but itis a fail
first time it enters in a test
first time it enters in a test
Saved previous test
first time it enters in a test
Saved previous test
first time it enters in a test
Saved previous test
first time it enters in a test
Saved previous test
first time it enters in a test
Saved previous test
Test ID :- SALE_MM_01
Test ID :- SALE_P_03
Test ID :- SALE_P_14
Test ID :- INSP_M_01
Test ID :- INSP_P_01
Test ID :- EXCL_M_01
Test ID :- EXCL_P_02


,Steps,Status,Start Time,End Time,test_names
0,101,fail,7:15:16 AM,7:21:40 AM,Test ID :- SALE_MM_01
1,119,pass,7:21:51 AM,7:28:45 AM,Test ID :- SALE_P_03
2,104,pass,7:28:58 AM,7:35:26 AM,Test ID :- SALE_P_14
3,104,pass,7:35:38 AM,7:42:13 AM,Test ID :- INSP_M_01
4,110,pass,7:42:22 AM,7:49:13 AM,Test ID :- INSP_P_01
5,54,pass,7:49:22 AM,7:51:19 AM,Test ID :- EXCL_M_01
6,56,pass,7:51:30 AM,7:53:28 AM,Test ID :- EXCL_P_02


In [12]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(open('C:\\Users\\c.hernandezperez\\Desktop\\drop\\reports\\342.html'), "html.parser")


In [19]:
test_names = [] 
for test in soup.find_all('span', {'class' : 'test-name'}):
    print(test.text)
    test_names.append(test.text)
print(test_names)

BT_AustraliaNewSubmission
BT_ChinaNewSubmissionRenewal
BT_HongKongNewSubmissionCopySubmission
BT_IndonesiaNewSubmission
BT_SingaporeNewSubmissionCancellation
BT_SpainNewSubmissionMultiversion
BT_SwitzerlandNewSubmissionChangePolicyEffDate
BT_AustraliaMta
['BT_AustraliaNewSubmission', 'BT_ChinaNewSubmissionRenewal', 'BT_HongKongNewSubmissionCopySubmission', 'BT_IndonesiaNewSubmission', 'BT_SingaporeNewSubmissionCancellation', 'BT_SpainNewSubmissionMultiversion', 'BT_SwitzerlandNewSubmissionChangePolicyEffDate', 'BT_AustraliaMta']
